# 02_preprocessing.ipynb
## 1. Imports & Load Raw Data

In [13]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [14]:


df = pd.read_csv("../archive/madrid_housing.csv")
print("Raw shape:", df.shape)
df.head()


Raw shape: (21742, 58)


,Unnamed: 0,id,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,...,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
0,0,21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,...,D,False,NaN,NaN,NaN,NaN,False,True,False,False
1,1,21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,21740,"Piso en venta en calle del Talco, 68","San Andrés, Madrid",94.0,54.0,2,2.0,NaN,NaN,...,no indicado,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,21739,Piso en venta en calle Pedro Jiménez,"San Andrés, Madrid",64.0,NaN,2,1.0,NaN,NaN,...,en trámite,False,NaN,NaN,NaN,NaN,False,False,True,False
4,4,21738,Piso en venta en carretera de Villaverde a Val...,"Los Rosales, Madrid",108.0,90.0,2,2.0,NaN,NaN,...,en trámite,True,NaN,NaN,True,0.0,True,True,True,True


## 2. handling missing values

In [15]:

missing_pct = df.isnull().mean().mul(100).sort_values(ascending=False)


display(missing_pct.head(20))          # 20 worst columns
print("Cols >80 % missing:", (missing_pct > 80).sum())


has_private_parking    100.000000
door                   100.000000
portal                 100.000000
is_furnished           100.000000
rent_price_by_area     100.000000
longitude              100.000000
latitude               100.000000
has_public_parking     100.000000
is_kitchen_equipped    100.000000
are_pets_allowed       100.000000
sq_mt_allotment         93.413669
n_floors                93.390672
has_garden              92.843345
has_balcony             84.725416
has_green_zones         81.340263
is_accessible           81.262073
has_pool                76.216539
street_number           71.023825
has_storage_room        64.593874
parking_price           64.497286
dtype: float64

Cols >80 % missing: 16


In [16]:
# Drop columns >80 % missing
cols_drop = missing_pct[missing_pct > 80].index.tolist()
df = df.drop(columns=cols_drop)
print("After col-drop:", df.shape)

# Drop rows with >50 % missing in remaining columns
row_thresh = df.shape[1] / 2
df = df[df.isnull().sum(axis=1) <= row_thresh].copy()
print("After row-drop:", df.shape)


After col-drop: (21742, 42)
After row-drop: (21707, 42)


In [17]:
drop_cols = ["street_name", "street_number"]
df = df.drop(columns=drop_cols)
print("After address drops:", df.shape)

After address drops: (21707, 40)


In [18]:
from sklearn.impute import SimpleImputer
import numpy as np

# sq_mt_useful 
def impute_useful(df):
    median_by_rooms = df.groupby("n_rooms")["sq_mt_useful"].transform("median")
    global_median   = df["sq_mt_useful"].median()
    df["sq_mt_useful"] = df["sq_mt_useful"].fillna(median_by_rooms).fillna(global_median)
    return df
df = impute_useful(df)

# floor 
mode_floor_by_sub = (
    df.groupby("subtitle")["floor"]
      .agg(lambda s: s.mode().iat[0] if not s.mode().empty else np.nan)
)
df["floor"] = df.apply(
    lambda row: mode_floor_by_sub[row["subtitle"]]
                if pd.isna(row["floor"]) else row["floor"],
    axis=1
)
df["floor"] = df["floor"].fillna(df["floor"].mode()[0])

# built_year
med_by_sub = df.groupby("subtitle")["built_year"].transform("median")
df["built_year"] = df["built_year"].fillna(med_by_sub).fillna(df["built_year"].median())

bool_cols = [
    "has_central_heating","has_individual_heating",
    "has_ac","has_fitted_wardrobes","has_pool",
    "has_terrace","has_storage_room",
    "is_parking_included_in_price",
    "is_orientation_north","is_orientation_west",
    "is_orientation_south","is_orientation_east"
]
for col in bool_cols:
    df[col + "_was_missing"] = df[col].isna()     # flag
    df[col] = df[col].fillna(False)

# parking_price 
df.loc[df["is_parking_included_in_price"], "parking_price"] = 0
df["parking_price"] = df["parking_price"].fillna(0)


print("Remaining NaNs:", df.isnull().sum().sort_values(ascending=False).head())


Remaining NaNs: raw_address           5430
is_exterior           3008
has_lift              2360
is_floor_under        1136
is_new_development     965
dtype: int64


C:\Users\Leon\AppData\Local\Temp\ipykernel_14280\3079625907.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False)
C:\Users\Leon\AppData\Local\Temp\ipykernel_14280\3079625907.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False)
C:\Users\Leon\AppData\Local\Temp\ipykernel_14280\3079625907.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavio

In [19]:

df["raw_address"] = df["raw_address"].fillna("missing")


bool_cols_remaining = ["is_exterior", "has_lift", "is_floor_under", "is_new_development"]

for col in bool_cols_remaining:
    df[col + "_was_missing"] = df[col].isna()   # optional flag
    df[col] = df[col].fillna(False)


print("Any NaNs left?", df.isnull().any().any())


Any NaNs left? True


C:\Users\Leon\AppData\Local\Temp\ipykernel_14280\1652675904.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False)
C:\Users\Leon\AppData\Local\Temp\ipykernel_14280\1652675904.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False)
C:\Users\Leon\AppData\Local\Temp\ipykernel_14280\1652675904.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, 

In [20]:
# Show columns that have more than 1 null value %
null_summary = (
    df.isnull().mean()
      .mul(100)
      .loc[lambda s: s > 0]
      .sort_values(ascending=False)
)

print("Columns still containing NaNs:")
display(null_summary)


Columns still containing NaNs:


house_type_id    1.801262
sq_mt_built      0.566637
n_bathrooms      0.073709
dtype: float64

In [21]:
# house_type_id  (categorical string)
df["house_type_id_was_missing"] = df["house_type_id"].isna()
mode_house_type = df["house_type_id"].mode()[0]
df["house_type_id"] = (
    df["house_type_id"]
      .fillna(mode_house_type)
      .astype("category")        # cast to pandas category, not int
)

# sq_mt_built  (numeric)
df["sq_mt_built"] = df["sq_mt_built"].fillna(df["sq_mt_built"].median())

# n_bathrooms  (numeric)
df["n_bathrooms"] = df["n_bathrooms"].fillna(df["n_bathrooms"].median())

# Sanity check
print("Any NaNs left? ->", df.isnull().any().any())


Any NaNs left? -> False


In [22]:
# Drop identifier / index columns
drop_cols = ["Unnamed: 0", "id"]              
df = df.drop(columns=[c for c in drop_cols if c in df.columns])


sale_mask = df["operation"].str.contains("sale|venta", na=False)
df.loc[sale_mask, "rent_price"] = 0                 
df["has_rent_price"] = ~sale_mask                  

# Clip impossible `built_year`
df["built_year_clipped"] = df["built_year"].clip(lower=1700, upper=2025)

# price_per_m2 and building_age
df["price_per_m2"] = df["buy_price"] / df["sq_mt_built"]
df["building_age"]  = 2025 - df["built_year_clipped"]

# Log-transform highly skewed columns
for col in ["buy_price", "sq_mt_built", "sq_mt_useful", "price_per_m2"]:
    df[f"log_{col}"] = np.log1p(df[col])

print("End shape :", df.shape)
df.head()

End shape : (21707, 63)


,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,raw_address,is_exact_address_hidden,floor,is_floor_under,...,is_new_development_was_missing,house_type_id_was_missing,has_rent_price,built_year_clipped,price_per_m2,building_age,log_buy_price,log_sq_mt_built,log_sq_mt_useful,log_price_per_m2
0,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,"Calle de Godella, 64",False,3,False,...,False,False,False,1960.0,1328.125000,65.0,11.350418,4.174387,4.110874,7.192276
1,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,82.0,3,1.0,Calle de la del Manojo de Rosas,True,4,False,...,False,False,False,1970.0,1855.714286,55.0,11.774528,4.262680,4.418841,7.526564
2,"Piso en venta en calle del Talco, 68","San Andrés, Madrid",94.0,54.0,2,2.0,"Calle del Talco, 68",False,1,False,...,False,False,False,1974.0,1534.542553,51.0,11.879289,4.553877,4.007333,7.336639
3,Piso en venta en calle Pedro Jiménez,"San Andrés, Madrid",64.0,63.0,2,1.0,Calle Pedro Jiménez,True,Bajo,True,...,False,False,False,1955.0,1717.187500,70.0,11.607335,4.174387,4.158883,7.449025
4,Piso en venta en carretera de Villaverde a Val...,"Los Rosales, Madrid",108.0,90.0,2,2.0,Carretera de Villaverde a Vallecas,True,4,False,...,False,False,False,2003.0,2407.407407,22.0,12.468441,4.691348,4.510860,7.786721


In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from category_encoders.target_encoder import TargetEncoder
import numpy as np

target = "log_buy_price"
y      = df[target].values

text_cols  = ["raw_address"]
bool_cols  = [c for c in df.columns if df[c].dtype == "bool"]

numeric_cols = df.select_dtypes(include=["number"]).columns.difference([target])
cat_cols     = df.select_dtypes(include=["object","category"]).columns.difference(text_cols)
low_card     = [c for c in cat_cols if df[c].nunique() <= 50]
high_card    = [c for c in cat_cols if df[c].nunique() > 50]

numeric_pipe   = Pipeline([("scaler", StandardScaler())])
low_card_pipe  = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore"))])
high_card_pipe = Pipeline([("target", TargetEncoder())])

# NEW: selector → TF-IDF
text_pipe = Pipeline([
    ("selector", FunctionTransformer(lambda x: x.squeeze(), validate=False)),
    ("tfidf",    TfidfVectorizer(max_features=20_000, ngram_range=(1,3)))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num",      numeric_pipe,  numeric_cols),
        ("bools",    "passthrough", bool_cols),
        ("low_cat",  low_card_pipe, low_card),
        ("high_cat", high_card_pipe, high_card),
        ("text",     text_pipe,     text_cols)   # keeps raw_address
    ],
    remainder="drop"
)

X = preprocessor.fit_transform(df, y)
print("Processed feature matrix shape:", X.shape)


Processed feature matrix shape: (21707, 20091)


In [24]:
df["listing_id"] = range(len(df))
df.to_csv("../data/processed/cleaned_data.csv", index=False)
